In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
mapping_home = {'Richmond ':'RCH','Geelong ':'GEE','North Melbourne ':'NTH','Port Adelaide ':'POR','Gold Coast ':'GCS','Melbourne ':'MEL','GWS ':'GWS','Hawthorn ':'HAW','Carlton ':'CAR','Brisbane ':'BRL','Collingwood ':'COL','Adelaide ':'ADE','Fremantle ':'FRE','Western Bulldogs ':'WBD','Sydney ':'SYD','West Coast ':'WCE','St Kilda ':'STK','Essendon ':'ESS'}
mapping_away = {' Richmond':'RCH', ' Geelong':'GEE', ' North Melbourne':'NTH', ' Port Adelaide':'POR', ' Gold Coast':'GCS', ' Melbourne':'MEL', ' GWS':'GWS', ' Hawthorn':'HAW', ' Carlton':'CAR', ' Brisbane':'BRL', ' Collingwood':'COL', ' Adelaide':'ADE', ' Fremantle':'FRE', ' Western Bulldogs':'WBD', ' Sydney':'SYD', ' West Coast':'WCE', ' St Kilda':'STK', ' Essendon':'ESS',}

In [24]:
tmp.tail(25)

,0,1,2,3,4
240,Sat 20 Aug 7:25pm,Essendon v Richmond,MCG,58366,75-141
241,Sun 21 Aug 1:10pm,Hawthorn v Western Bulldogs,UTAS Stadium,13105,64-87
242,Sun 21 Aug 3:20pm,Carlton v Collingwood,MCG,88287,74-75
243,Sun 21 Aug 4:40pm,St Kilda v Sydney,Marvel Stadium,23344,74-88
244,Elimination Final,Elimination Final,Elimination Final,Elimination Final,Elimination Final
245,Date,Home v Away Teams,Venue,Crowd,Result
246,Thu 1 Sep 7:20pm,Brisbane v Richmond,Gabba,35013,106-104
247,Qualifying Final,Qualifying Final,Qualifying Final,Qualifying Final,Qualifying Final
248,Date,Home v Away Teams,Venue,Crowd,Result
249,Fri 2 Sep 7:50pm,Melbourne v Sydney,MCG,78377,69-91


In [28]:
finals = ['Elimination Final','Qualifying Final','Semi Final','Preliminary Final','Grand Final']

In [41]:
lst = list(tmp[tmp[0].str.contains('Round *')].index)
for final in finals:
    lst = lst + list(tmp[tmp[0].str.contains(final)].index)
lst.append(tmp.shape[0])

In [44]:
url = f'https://www.footywire.com/afl/footy/ft_match_list?year=2022'
stat_tables = pd.read_html(url)
tmp = stat_tables[9].drop(columns=[5,6]).reset_index(drop = True)
tmp = tmp[tmp[0].notna()].fillna('').reset_index(drop = True)

lst = list(tmp[tmp[0].str.contains('Round *')].index)
for final in finals:
    lst = lst + list(tmp[tmp[0].str.contains(final)].index)
lst.append(tmp.shape[0])
r = 1
output = pd.DataFrame({})
for index in range(len(lst) - 1):
    try:
        tmp1 = tmp.iloc[lst[index]+1: lst[index + 1]]
        headers = list(tmp1.iloc[0])

        tmp1  = pd.DataFrame(tmp1.values[1:], columns=headers)
        tmp1['round'] = f'R{r}'

        r += 1

        output = pd.concat([output,tmp1])
    except:
        continue

In [45]:
output

,Date,Home v Away Teams,Venue,Crowd,Result,round
0,Wed 16 Mar 7:10pm,Melbourne v Western Bulldogs,MCG,58002,97-71,R1
1,Thu 17 Mar 7:25pm,Carlton v Richmond,MCG,72179,101-76,R1
2,Fri 18 Mar 7:50pm,St Kilda v Collingwood,Marvel Stadium,40129,85-102,R1
3,Sat 19 Mar 2:10pm,Geelong v Essendon,MCG,54495,138-72,R1
4,Sat 19 Mar 5:10pm,GWS v Sydney,Accor Stadium,25572,92-112,R1
...,...,...,...,...,...,...
0,Fri 9 Sep 7:50pm,Melbourne v Brisbane,MCG,62162,79-92,R26
1,Sat 10 Sep 7:25pm,Collingwood v Fremantle,MCG,90612,79-59,R26
0,Fri 16 Sep 7:50pm,Geelong v Brisbane,MCG,77677,120-49,R27
1,Sat 17 Sep 4:45pm,Sydney v Collingwood,SCG,45608,95-94,R27


In [7]:
tmp

,0,1,2,3,4
0,Round 1,Round 1,Round 1,Round 1,Round 1
1,Date,Home v Away Teams,Venue,Crowd,Result
2,Wed 16 Mar 7:10pm,Melbourne v Western Bulldogs,MCG,58002,97-71
3,Thu 17 Mar 7:25pm,Carlton v Richmond,MCG,72179,101-76
4,Fri 18 Mar 7:50pm,St Kilda v Collingwood,Marvel Stadium,40129,85-102
...,...,...,...,...,...
260,Fri 16 Sep 7:50pm,Geelong v Brisbane,MCG,77677,120-49
261,Sat 17 Sep 4:45pm,Sydney v Collingwood,SCG,45608,95-94
262,Grand Final,Grand Final,Grand Final,Grand Final,Grand Final
263,Date,Home v Away Teams,Venue,Crowd,Result


In [3]:
for year in range(2023, 2024):
    url = f'https://www.footywire.com/afl/footy/ft_match_list?year={year}'
    stat_tables = pd.read_html(url)
    tmp = stat_tables[9].drop(columns=[5,6]).reset_index(drop = True)
    if year == 2020:
        tmp = tmp[tmp[0].notna()].reset_index(drop = True)
    else:
        tmp = tmp[tmp[0].notna()].fillna('').reset_index(drop = True)
    
    lst = list(tmp[tmp[0].str.contains('Round *')].index)
    lst.append(tmp.shape[0])

    r = 1
    output = pd.DataFrame({})
    for index in range(len(lst) - 1):
        try:
            tmp1 = tmp.iloc[lst[index]+1: lst[index + 1]]
            headers = list(tmp1.iloc[0])

            tmp1  = pd.DataFrame(tmp1.values[1:], columns=headers)
            tmp1['round'] = f'R{r}'

            r += 1

            output = pd.concat([output,tmp1])
        except:
            continue
    
    output['year'] = year
    output['Home'] = output['Home v Away Teams'].apply(lambda x: x.split('v')[0])
    output['Away'] = output['Home v Away Teams'].apply(lambda x: x.split('v')[1])
    
    output['Home Score'] = output['Result'].apply(lambda x: x.split('-')[0] if x!='' else '')
    
    output['Away Score'] = output['Result'].apply(lambda x: x.split('-')[1] if x!='' else '')
    
    
    output = output.drop(columns = ['Result','Home v Away Teams'])  
    for key in mapping_home.keys():
        output.loc[output['Home']==key, 'Home'] = mapping_home[key]
    for key in mapping_away.keys():
        output.loc[output['Away']==key, 'Away'] = mapping_away[key]
    
    output.to_csv(f'/Users/caseyh/Desktop/footyfootyfooty/round_results/{year}_round_results.csv', index = False)